# NPS adjustments

Before trainning the model, there are a few caveats about the NPS and the touchpoints' data. Particularly, there are 3 adjustments we have to consider:
1) Monthly statistical adjustment (weights): Each month, the NPS scores per cabin/haul are adjusted to fit a true sample of the global distribution of cabin/haul users. That is, for the NPS per cabin haul to be representative, it is first adjusted so that the sample of users from which it is computed is representative as well. That way, if by chance a month has only one passenger in, for example, Bussines-Long Haul, the associated NPS considers that that particular sample is not representtive of the normal distribution of Business Long Haul users. Unfortunatelly, this values are computed every month, and we wont have access to them at prediction time. 
2) Change of survey: On early december 2019 and on late december 2023 there have been changes on the survey handled to the clients that have considerelly affected the NPS score and the satisfaction per touchpoint. A fixed value (per cabin/haul and globally) is used to adjust 2019 and 2022 (plus december 2019). Those values are computed externally.
3) C/H ponderation: On the global NPS computation, each cabin haul NPS is adjusted by a factor based on the percentaje of users that will fly on that particular cabine/haul. These values are fixed and at the moment of prediction we will have access to them.

## Installs and imports

In [2]:
!pip install plotly
!pip install boto3==1.19.12
!pip install s3fs


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# General
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import os
import numpy as np
import xlsxwriter
import datetime
import boto3


# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

# Plots
import matplotlib.pyplot as plt
import seaborn as sns

#Warnings
import warnings
warnings.filterwarnings("ignore")

## Read files

In [4]:
nps_path = 's3://iberia-data-lake/customer/nps_surveys/export_historic/insert_date_ci=2023-11-21/'
iberia_kpis_path = 's3://iberia-data-lake/customer/one_shot/iberia_kpis_daily/'

In [5]:
import boto3
import pandas as pd

def read_csv_files_from_s3(s3_path):
    # Initialize Boto3 client for S3
    s3_client = boto3.client('s3')

    # Extract bucket name and prefix from the S3 path
    bucket, prefix = s3_path.replace('s3://', '').split('/', 1)

    # Get a list of all keys (object paths) under the specified S3 path
    s3_resource = boto3.resource('s3')
    s3_keys = [item.key for item in s3_resource.Bucket(bucket).objects.filter(Prefix=prefix)]

    # Generate a list of full S3 paths for all CSV files
    preprocess_paths = [f"s3://{bucket}/{key}" for key in s3_keys]

    # Read all CSV files into a list of DataFrames
    dfs = [pd.read_csv(s3_client.get_object(Bucket=bucket, Key=key)['Body']) for key in s3_keys]

    # Concatenate DataFrames into a single DataFrame
    df_concatenated = pd.concat(dfs, axis=0, ignore_index=True)

    return df_concatenated

In [6]:
# Readd files from paths
df_nps = read_csv_files_from_s3(nps_path)

In [7]:
df_nps[df_nps['monthly_weight'].notna()]['date_flight_local'].max()

'2023-10-31'

In [8]:
df_kpis = read_csv_files_from_s3(iberia_kpis_path)

In [9]:
df_kpis[df_kpis['year']==2023]


,year,month,day,haul,cabin,load_factor,otp15_landing,otp15_takeoff
3,2023,10,11,LH,Premium Economy,80.700,82.759,81.034
5,2023,2,9,LH,Premium Economy,79.330,91.304,95.652
20,2023,7,29,LH,Premium Economy,89.551,86.667,90.000
27,2023,5,8,LH,Premium Economy,87.931,89.286,92.857
51,2023,5,7,SH,Premium Economy,75.000,92.290,92.523
...,...,...,...,...,...,...,...,...
17780,2023,2,6,SH,Business,66.553,93.842,92.365
17781,2023,7,23,SH,Business,73.243,83.102,85.185
17787,2023,1,6,SH,Business,51.168,91.123,91.123
17801,2023,4,5,SH,Business,61.013,86.667,88.642


## 1) Monthly statistical adjustment.

In [10]:
condition_1 = (df_nps['operating_airline_code'].isin(['IB', 'YW']))
condition_2 = ((df_nps['invitegroup_ib'] != 3) | (df_nps['invitegroup_ib'].isnull()))
condition_3 = (df_nps['invitegroup'] == 2)

df_nps_tkt = df_nps.loc[condition_1 & (condition_2 & condition_3)]

In [11]:
df_nps

,respondent_id,sample_id,surveyed_flight_number,date_flight_local,scheduled_departure_time_local,scheduled_arrival_time_local,tier_level,language_code,aircraft_registration_number,seat_no,volume_of_bags,number_of_child_in_the_booking,number_of_infant_in_the_booking,number_of_people_in_the_booking,infinita_customers_identifer,flag_of_ib_singular_customers,country_code,list_of_options_for_booking_channel,list_of_options_for_checkin_channel,lounge_used_at_origin_airport,customer_journey_origin,customer_journey_destination,number_of_flights_in_journey,order_of_flight_in_journey,fleet_in_surveyed_flight,marketing_airline_code,date_of_flight_gmt,scheduled_departure_time_gmt,real_departure_time_local,real_departure_time_gmt,scheduled_arrival_time_gmt,real_arrival_time_local,real_arrival_time_gmt,segment,route,overall_haul,purser,invitegroup_ib,weight_category,weekly_weight,monthly_weight,pnr_show,ff_number,id_golden_record,ticket_num,started,time_spent_hrminsec,customer_email_show,origin_of_surveyed_flight,destination_of_surveyed_flight,operating_airline_code,cabin_in_surveyed_flight,haul,first_name_show,last_name_show,second_last_name_show,nps_category,nps_100,survey_type,invitegroup,group_age_survey,pun_100_punctuality,inm_200_issues_prior_checkin,inm_200_issues_prior_ticket_change,inm_200_issues_prior_schedule_change,inm_200_issues_prior_contact_center,inm_200_issues_prior_special_serv,inm_200_issues_prior_special_req,inm_200_issues_prior_avios,inm_200_issues_prior_voucher,inm_206_issues_checkin_long_queues,inm_206_issues_checkin_wrong_info,inm_206_issues_checkin_additional_fees,inm_206_issues_checkin_overbooking,inm_206_issues_checkin_downgrade,inm_206_issues_checkin_staff,inm_206_issues_checkin_social_distance,inm_206_issues_checkin_face_masks,inm_206_issues_checkin_documentation,inm_206_issues_checkin_other,inm_207_issues_lounge_denied,inm_207_issues_lounge_overcrowded,inm_207_issues_lounge_cleanliness,inm_207_issues_lounge_wifi,inm_207_issues_lounge_staff,inm_207_issues_lounge_food_drink,inm_207_issues_lounge_face_masks,inm_207_issues_lounge_other,inm_208_issues_security_leave_sth,inm_208_issues_security_long_queues,inm_208_issues_security_staff,inm_208_issues_security_social_distance,inm_208_issues_security_face_masks,inm_208_issues_security_other,inm_209_issues_boarding_unclear,inm_209_issues_boarding_gate_changed,inm_209_issues_boarding_lack_space,inm_209_issues_boarding_disorganised,inm_209_issues_boarding_staff,inm_209_issues_boarding_social_distance,inm_209_issues_boarding_face_masks,inm_209_issues_boarding_documentation,inm_209_issues_boarding_other,inm_220_issues_timing_cancelled,inm_220_issues_timing_dep_delay,inm_220_issues_timing_arr_delay,inm_220_issues_timing_missed,inm_230_issues_onboard_staff,inm_230_issues_onboard_ife,inm_230_issues_onboard_overcrowding,inm_230_issues_onboard_face_masks,inm_235_issues_onboard_comfort_damaged,inm_235_issues_onboard_comfort_space,inm_235_issues_onboard_comfort_temperature,inm_235_issues_onboard_comfort_cleanliness,inm_235_issues_onboard_comfort_washrooms,inm_235_issues_onboard_comfort_other,inm_236_issues_meal_availability,inm_236_issues_meal_portions,inm_236_issues_meal_quality,inm_236_issues_meal_special,inm_236_issues_meal_other,inm_240_issues_baggage_lost,inm_240_issues_baggage_delayed,inm_240_issues_baggage_demaged,inm_240_issues_baggage_staff,inm_240_issues_baggage_hand,inm_240_issues_baggage_other,inm_250_issues_arrival_slow,inm_250_issues_arrival_unclear,inm_250_issues_arrival_aditional_request,inm_250_issues_arrival_staff,inm_250_issues_arrival_immigration_queues,inm_250_issues_arrival_immigration_passport,inm_250_issues_arrival_immigration_other,inm_255_issues_connecting_missed,inm_255_issues_connecting_staff,inm_255_issues_connecting_baggage,inm_255_issues_connecting_other,bkg_100_booking,bkg_200_journey_preparation,inm_400_issues_response,pfl_100_checkin,pfl_200_security,pfl_300_lounge,pfl_500_boarding,ifl_100_cabin_crew,ifl_200_flight_crew_annoucements,ifl_300_cabin,

In [12]:
datetime_features = ['date_flight_local', 'scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local',
                     'real_arrival_time_local', 'started']
columns_to_cross_kpis=['cabin_in_surveyed_flight','haul']
columns_ext = ['tier_level', 'language_code', 'seat_no', 'volume_of_bags', 'number_of_child_in_the_booking', 'number_of_infant_in_the_booking',
              'number_of_people_in_the_booking', 'country_code', 'customer_journey_origin', 'customer_journey_destination', 'number_of_flights_in_journey',
              'order_of_flight_in_journey', 'marketing_airline_code', 'overall_haul', 'weight_category', 'ff_number', 'ticket_num', 'operating_airline_code',
               'nps_category', 'nps_100', 'group_age_survey', 'gender'] # invite_group

#'bkg_100_booking', 
touchpoints = ['bkg_200_journey_preparation', 'pfl_100_checkin', 'pfl_200_security', 'pfl_300_lounge',
               'pfl_500_boarding', 'ifl_300_cabin', 'ifl_200_flight_crew_annoucements', 'ifl_600_wifi', 'ifl_500_ife',
               'ifl_400_food_drink', 'ifl_100_cabin_crew', 'arr_100_arrivals', 'con_100_connections', 'pun_100_punctuality',
               'loy_200_loyalty_programme', 'inm_400_issues_response', 'img_310_ease_contact_phone']

# ,'img_320_ease_contact_ibplus_mail'
survey_fields = ['cla_600_wifi_t_f', 'tvl_journey_reason']

for feat in datetime_features:
    if feat in ['scheduled_departure_time_local', 'scheduled_arrival_time_local', 'real_departure_time_local', 'real_arrival_time_local','date_flight_local']:
        df_nps_tkt[feat] = pd.to_datetime(df_nps_tkt[feat], format="%Y%m%d %H:%M:%S", errors = 'coerce')
    else:
        df_nps_tkt[feat] = pd.to_datetime(df_nps_tkt[feat], errors = 'ignore')
df_nps_tkt['time_spent_hrminsec'] = pd.to_timedelta(df_nps_tkt['time_spent_hrminsec']).dt.total_seconds()
df_nps_tkt['started_hour'] = df_nps_tkt['started'].dt.hour
df_nps_tkt['year_flight'] = df_nps_tkt['date_flight_local'].dt.year
df_nps_tkt['month_flight'] = df_nps_tkt['date_flight_local'].dt.month
df_nps_tkt['day_flight'] = df_nps_tkt['date_flight_local'].dt.day
df_nps_tkt['weekday_flight'] = df_nps_tkt['date_flight_local'].dt.weekday
df_nps_tkt['is_weekend_or_friday_flight'] = df_nps_tkt['weekday_flight'].apply(lambda x: 1 if x in [5, 6,7] else 0)
df_nps_tkt['delay_departure'] = (df_nps_tkt['real_departure_time_local'] - df_nps_tkt['scheduled_departure_time_local']).dt.total_seconds()/60
df_nps_tkt['delay_arrival'] = (df_nps_tkt['real_arrival_time_local'] - df_nps_tkt['scheduled_arrival_time_local']).dt.total_seconds()/60
datetime_features = datetime_features + ['time_spent_hrminsec', 'started_hour', 'year_flight', 'month_flight',
                                         'day_flight', 'weekday_flight', 'is_weekend_or_friday_flight']

In [13]:
df_nps_tkt=df_nps_tkt[df_nps_tkt['date_flight_local'].dt.year >= 2019]

In [14]:
df_nps_tkt['date_flight_local'].min()

Timestamp('2019-01-01 00:00:00')

In [15]:
columns_to_select = datetime_features + columns_ext + touchpoints + ['delay_arrival', 'delay_departure', 'ticket_price']+columns_to_cross_kpis+['monthly_weight']

In [16]:
df_nps_tkt = df_nps_tkt[columns_to_select]
df_nps_tkt[['date_flight_local','nps_100','monthly_weight']]

,date_flight_local,nps_100,monthly_weight
1,2023-01-04,8,0.519
2,2023-01-29,10,0.414
3,2019-07-23,9,1.593
4,2019-09-03,6,0.729
6,2019-02-16,9,1.502
...,...,...,...
915849,2023-11-16,10,NaN
915850,2023-10-05,9,0.473
915851,2023-09-07,9,0.476
915852,2023-10-12,10,0.367


In [17]:
x=df_nps_tkt.copy()
pd.set_option('display.max_rows', 100)
x = x[(x['date_flight_local']=='2022-12-01') & (x['cabin_in_surveyed_flight'] == 'Economy') & (x['haul'] == 'MH') ]
display(x[(x['nps_100'] > 8) | (x['nps_100'] < 6)][['date_flight_local','cabin_in_surveyed_flight','haul','monthly_weight','nps_100']].head(200))



,date_flight_local,cabin_in_surveyed_flight,haul,monthly_weight,nps_100
14391,2022-12-01,Economy,MH,2.708,9
27782,2022-12-01,Economy,MH,2.708,10
54798,2022-12-01,Economy,MH,2.708,9
66610,2022-12-01,Economy,MH,2.708,9
66624,2022-12-01,Economy,MH,2.708,3
68278,2022-12-01,Economy,MH,2.708,5
68419,2022-12-01,Economy,MH,2.708,10
68450,2022-12-01,Economy,MH,2.708,5
90665,2022-12-01,Economy,MH,2.708,10
90863,2022-12-01,Economy,MH,2.708,10


#### If 4 cabin/haul

Before aggregating, I need to rewrite come categories in order to comapre with the dashboard:

In [18]:
df_nps_tkt['haul'] = df_nps_tkt['haul'].replace('MH', 'SH')
#df_nps_tkt['cabin_in_surveyed_flight'] = df_nps_tkt['cabin_in_surveyed_flight'].replace('Premium Economy', 'Economy')

#df_kpis['cabin']=df_kpis['cabin'].replace('Premium Economy','Premium Economy')
df_kpis['cabin']=df_kpis['cabin'].replace('Premium','Business')


In [19]:
df_nps_tkt[df_nps_tkt['monthly_weight'].isnull()][['date_flight_local','cabin_in_surveyed_flight','haul']]

,date_flight_local,cabin_in_surveyed_flight,haul
32769,2023-06-30,Economy,SH
33783,2023-07-31,Premium Economy,LH
33927,2023-07-31,Economy,SH
34934,2023-11-12,Premium Economy,LH
34937,2023-11-02,Economy,LH
...,...,...,...
915814,2023-11-05,Economy,SH
915823,2023-11-14,Economy,SH
915844,2023-11-10,Economy,LH
915849,2023-11-16,Economy,LH


In [20]:
df_nps_tkt = df_nps_tkt[df_nps_tkt['monthly_weight'].notnull()]

### Aggregate on time frequency

In [21]:
# Function to calculate NPS
def calculate_nps(promoters, detractors, passives):
    total_responses = promoters + detractors + passives
    nps = 100 * (promoters - detractors) / total_responses
    return nps

def calculate_aggregated_features_with_cross_kpis(df, variables, columns_to_cross_kpis, time_frequency='D', calculate_satisfaction=True):
    touchpoints = [var for var in variables if calculate_satisfaction]
    non_touchpoints = [var for var in variables if not calculate_satisfaction]

    # Combine the values of columns_to_cross_kpis into a new column 'cross_kpis'
    df['cross_kpis'] = df[columns_to_cross_kpis].apply(lambda x: '_'.join(x.values.astype(str)), axis=1)

    # Group the data by 'cross_kpis' and 'date_flight_local' using pd.Grouper on the entire DataFrame
    grouped_data = df.groupby([pd.Grouper(key='cross_kpis'), pd.Grouper(key='date_flight_local', freq=time_frequency)], group_keys=False)

    # Calculate the aggregated features for each group
    aggregated_features_data = pd.DataFrame()
    
    # Calculate the aggregated features for each group
    aggregated_features_data['NPS'] = grouped_data.apply(lambda x: calculate_nps(
        sum((x['nps_100'] == 9) | (x['nps_100'] == 10)),
        sum(x['nps_100'] <= 6),
        sum((x['nps_100'] == 7) | (x['nps_100'] == 8))
    ))

    # Calculate promoters_weight and detractors_weight
    aggregated_features_data['promoters_weight'] = grouped_data.apply(lambda x: (x[x['nps_100'] > 8]['monthly_weight'].sum()) if (x['nps_100'] > 8).any() else 0)
    aggregated_features_data['detractors_weight'] = grouped_data.apply(lambda x: (x[x['nps_100'] <= 6]['monthly_weight'].sum()) if (x['nps_100'] <= 6).any() else 0)




    # Calculate monthly_weight within each group
    aggregated_features_data['monthly_weight'] = grouped_data.apply(lambda x: x['monthly_weight'].sum())

    # Calculate NPS_weighted using the same logic as your SQL query
    aggregated_features_data['NPS_weighted'] = (aggregated_features_data['promoters_weight'] - aggregated_features_data['detractors_weight']) * 100 / aggregated_features_data['monthly_weight']

    for col in touchpoints + non_touchpoints:
        aggregated_features_data[f'{col}_sum'] = grouped_data[col].sum()
        aggregated_features_data[f'{col}_std'] = grouped_data[col].std()
        aggregated_features_data[f'{col}_mean'] = grouped_data[col].mean()
        aggregated_features_data[f'{col}_not_nulls'] = grouped_data[col].apply(lambda x: x.notnull().sum())
        if col in touchpoints:
            aggregated_features_data[f'{col}_satisfaction'] = grouped_data[col].apply(lambda x: x[x >= 8].count() / x.count()*100)
        else:
            aggregated_features_data[f'{col}_max'] = grouped_data[col].max()
            aggregated_features_data[f'{col}_min'] = grouped_data[col].min()
    
    # Reset the index of aggregated_features_data
    aggregated_features_data.reset_index(inplace=True)

    # Split the 'cross_kpis' column back into individual columns and concatenate them to aggregated_features_data
    split_columns = aggregated_features_data['cross_kpis'].str.split('_', expand=True)
    split_columns.columns = columns_to_cross_kpis
    aggregated_features_data = pd.concat([split_columns, aggregated_features_data], axis=1)
    
    # Drop the temporary 'cross_kpis' column
    aggregated_features_data.drop(columns=['cross_kpis'], inplace=True)
    
    return aggregated_features_data


In [22]:
df=calculate_aggregated_features_with_cross_kpis(df_nps_tkt, touchpoints, columns_to_cross_kpis, time_frequency='D', calculate_satisfaction=True)

In [23]:
df.describe()

,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction
count,8411.000,8411.000,8411.000,8411.000,8411.000,8411.000,8159.000,8386.000,8411.000,8386.000,8411.000,8208.000,8409.000,8411.000,8409.000,8411.000,8205.000,8408.000,8411.000,8408.000,8411.000,6482.000,7397.000,8411.000,7397.000,8411.000,8214.000,8410.000,8411.000,8410.000,8411.000,8214.000,8411.000,8411.000,8411.000,8411.000,8203.000,8406.000,8411.000,8406.000,8411.000,5911.000,6928.000,8411.000,6928.000,8411.000,6749.000,7510.000,8411.000,7510.000,8411.000,8149.000,8368.000,8411.000,8368.000,8411.000,8214.000,8408.000,8411.000,8408.000,8411.000,8197.000,8406.000,8411.000,8406.000,8411.000,6465.000,7443.000,8411.000,7443.000,8411.000,8217.000,8411.000,8411.000,8411.000,8411.000,7670.000,8102.000,8411.000,8102.000,8411.000,5169.000,6653.000,8411.000,6653.000,8411.000,8065.000,8350.000,8411.000,8350.000
mean,31.747,33.476,14.484,62.664,31.745,442.261,2.584,7.564,58.923,66.262,529.093,2.261,7.990,66.507,73.111,546.710,1.928,8.245,66.130,76.296,58.476,2.041,7.892,7.375,69.952,529.665,2.148,7.901,67.226,70.297,534.681,2.036,8.022,67.425,71.952,547.105,1.877,8.333,65.967,76.716,71.881,3.313,5.063,13.178,32.492,142.839,2.301,7.432,18.411,63.518,303.341,2.593,6.842,44.932,51.996,549.844,2.087,8.204,66.328,75.327,530.944,2.066,7.985,66.388,71.801,148.177,2.576,7.393,20.147,64.072,561.978,2.142,8.543,68.241,82.105,222.550,2.132,7.435,30.313,58.754,26.193,3.050,3.187,8.950,14.632,191.078,3.038,6.163,31.555,44.233
std,30.390,51.264,22.810,95.126,30.392,707.153,0.883,1.015,93.386,17.158,861.060,0.838,0.885,106.931,15.832,885.088,0.730,0.748,106.503,15.124,61.003,1.083,1.323,7.607,25.173,853.574,0.752,0.876,107.995,16.

### Merge dfs

In [24]:
# Convert the 'date_flight_local' column in result_df to separate 'year' and 'month' columns
df['year'] = df['date_flight_local'].dt.year
df['month'] = df['date_flight_local'].dt.month
df['day'] = df['date_flight_local'].dt.day
# Perform the merge based on the specified columns
merged_df = df.merge(df_kpis, 
                            left_on=['year', 'month', 'day','cabin_in_surveyed_flight', 'haul'], 
                            right_on=['year', 'month','day', 'cabin', 'haul'], 
                            how='inner')

# Drop the redundant columns from the merged dataframe
merged_df.drop(columns=['cabin_in_surveyed_flight'], inplace=True)

# Reorder the columns to place 'year', 'month', 'cabin', and 'haul' at the beginning
cols_to_move = ['date_flight_local','year', 'month', 'day', 'cabin', 'haul']
merged_df = merged_df[cols_to_move + [col for col in merged_df.columns if col not in cols_to_move]]
merged_df.head()

,date_flight_local,year,month,day,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
0,2019-01-01,2019,1,1,Business,LH,100.000,1.534,0.000,1.534,100.000,24.000,1.000,8.000,3,66.667,22.000,1.528,7.333,3,33.333,16.000,4.041,5.333,3,33.333,5.000,3.536,2.500,2,0.000,23.000,1.155,7.667,3,33.333,25.000,0.577,8.333,3,100.000,26.000,0.577,8.667,3,100.000,8.000,NaN,8.000,1,100.000,14.000,2.828,7.000,2,50.000,25.000,1.155,8.333,3,66.667,27.000,1.000,9.000,3,100.000,18.000,0.000,9.000,2,100.000,0.000,NaN,NaN,0,NaN,29.000,0.577,9.667,3,100.000,23.000,1.528,7.667,3,66.667,0.000,NaN,NaN,0,NaN,19.000,1.528,6.333,3,33.333,61.132,82.979,76.596
1,2019-01-01,2019,1,1,Business,LH,100.000,1.534,0.000,1.534,100.000,24.000,1.000,8.000,3,66.667,22.000,1.528,7.333,3,33.333,16.000,4.041,5.333,3,33.333,5.000,3.536,2.500,2,0.000,23.000,1.155,7.667,3,33.333,25.000,0.577,8.333,3,100.000,26.000,0.577,8.667,3,100.000,8.000,NaN,8.000,1,100.000,14.000,2.828,7.000,2,50.000,25.000,1.155,8.333,3,66.667,27.000,1.000,9.000,3,100.000,18.000,0.000,9.000,2,100.000,0.000,NaN,NaN,0,NaN,29.000,0.577,9.667,3,100.000,23.000,1.528,7.667,3,66.667,0.000,NaN,NaN,0,NaN,19.000,1.528,6.333,3,33.333,61.132,82.979,76.596
2,2019-01-02,2019,1,2,Business,LH,30.000,3.067,1.534,5.112,30.000,71.000,2.713,7.889,9,55.556,90.000,1.491,9.000,10,80.000,73.000,1.126,9.125,8,87.500,63.000,2.872,7.000,9,66.667,89.000,1.287,8.900,10,80.000,82.000,2.440,8.200,10,80.000,77.000,1.590,8.556,9,88.889,12.000,2.160,3.000,4,0.000,54.000,3.000,6.000,9,33.333,69.000,3.071,6.900,10,50.000,95.000,0.850,9.500,10,100.000,81.000,1.663,8.100,10,60.000,7.000,NaN,7.000,1,0.000,90.000,1.563,9.000,

In [25]:
merged_df.describe()

,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
count,16816.000,16816.000,16816.000,16816.000,16816.000,16816.000,16318.000,16766.000,16816.000,16766.000,16816.000,16416.000,16812.000,16816.000,16812.000,16816.000,16410.000,16810.000,16816.000,16810.000,16816.000,12964.000,14794.000,16816.000,14794.000,16816.000,16428.000,16814.000,16816.000,16814.000,16816.000,16428.000,16816.000,16816.000,16816.000,16816.000,16406.000,16806.000,16816.000,16806.000,16816.000,11822.000,13852.000,16816.000,13852.000,16816.000,13498.000,15014.000,16816.000,15014.000,16816.000,16298.000,16730.000,16816.000,16730.000,16816.000,16428.000,16810.000,16816.000,16810.000,16816.000,16394.000,16808.000,16816.000,16808.000,16816.000,12930.000,14884.000,16816.000,14884.000,16816.000,16434.000,16816.000,16816.000,16816.000,16816.000,15340.000,16204.000,16816.000,16204.000,16816.000,10338.000,13302.000,16816.000,13302.000,16816.000,16130.000,16698.000,16816.000,16698.000,16816.000,16816.000,16816.000
mean,31.770,33.488,14.489,62.686,31.769,442.417,2.584,7.565,58.944,66.273,529.279,2.261,7.991,66.530,73.126,546.902,1.928,8.245,66.153,76.300,58.497,2.041,7.892,7.377,69.952,529.851,2.148,7.901,67.250,70.298,534.869,2.036,8.022,67.449,71.965,547.298,1.877,8.333,65.990,76.731,71.906,3.313,5.063,13.182,32.502,142.888,2.301,7.433,18.417,63.517,303.448,2.593,6.843,44.947,52.003,550.038,2.087,8.204,66.351,75.330,531.132,2.066,7.985,66.411,71.806,148.229,2.576,7.392,20.154,64.067,562.177,2.142,8.545,68.265,82.122,222.630,2.132,7.435,30.324,58.754,26.202,3.050,3.187,8.953,14.637,191.145,3.038,6.163,31.566,44.226,72.641,85.912,87.762
std,30.317,51.268,22.812,

In [26]:
merged_df['date_flight_local'].max()

Timestamp('2023-10-31 00:00:00')

## 2) Change of survey adjustment.

Since the adjustment is done separatelly in the global NPS and in the Cabin/haul NPS we need to have a separate dataframe for the NPS.  

In [27]:
# Sample adjustments DataFrame
adjustments_data = {'haul': ['LH', 'LH', 'LH', 'SH', 'SH', 'SH'],
                    'cabin': ['Economy', 'Premium Economy' , 'Business', 'Economy', 'Premium Economy', 'Business'],
                    'adjustment_2019': [-0.55475719, -0.55475719, -6.86337594, -5.25927601, -5.25927601, -5.64891133],
                    'adjustment_2022': [1.41878561, 1.41878561, 0.42510779, -3.71320153, -3.71320153, -4.10337349]}

adjustments_df = pd.DataFrame(adjustments_data)
df=merged_df.copy()

for index, row in adjustments_df.iterrows():
    condition = ((df['haul'] == row['haul']) | (df['haul'] == 'MH')) & (df['cabin'] == row['cabin'])
    
    mask = (df['date_flight_local'].dt.year == 2019) & (df['date_flight_local'].dt.month != 12)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2019']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2019']
    
    mask = (df['date_flight_local'].dt.year == 2019) & (df['date_flight_local'].dt.month == 12)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2022']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2022']
    
    mask = (df['date_flight_local'].dt.year == 2022)
    df.loc[condition & mask, 'NPS'] += row['adjustment_2022']
    df.loc[condition & mask, 'NPS_weighted'] += row['adjustment_2022']
    
df[df['date_flight_local'] == '2022-12-31'].head()


,date_flight_local,year,month,day,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
2716,2022-12-31,2022,12,31,Business,LH,100.425,1.959,0.000,1.959,100.425,9.000,NaN,9.000,1,100.000,9.000,NaN,9.000,1,100.000,9.000,NaN,9.000,1,100.000,9.000,NaN,9.000,1,100.000,9.000,NaN,9.000,1,100.000,10.000,NaN,10.000,1,100.000,10.000,NaN,10.000,1,100.000,0.000,NaN,NaN,0,NaN,10.000,NaN,10.000,1,100.000,10.000,NaN,10.000,1,100.000,10.000,NaN,10.000,1,100.000,7.000,NaN,7.000,1,0.000,0.000,NaN,NaN,0,NaN,10.000,NaN,10.000,1,100.000,9.000,NaN,9.000,1,100.000,0.000,NaN,NaN,0,NaN,0.000,NaN,NaN,0,NaN,55.476,78.846,94.231
2717,2022-12-31,2022,12,31,Business,LH,100.425,1.959,0.000,1.959,100.425,9.000,NaN,9.000,1,100.000,9.000,NaN,9.000,1,100.000,9.000,NaN,9.000,1,100.000,9.000,NaN,9.000,1,100.000,9.000,NaN,9.000,1,100.000,10.000,NaN,10.000,1,100.000,10.000,NaN,10.000,1,100.000,0.000,NaN,NaN,0,NaN,10.000,NaN,10.000,1,100.000,10.000,NaN,10.000,1,100.000,10.000,NaN,10.000,1,100.000,7.000,NaN,7.000,1,0.000,0.000,NaN,NaN,0,NaN,10.000,NaN,10.000,1,100.000,9.000,NaN,9.000,1,100.000,0.000,NaN,NaN,0,NaN,0.000,NaN,NaN,0,NaN,55.476,78.846,94.231
6114,2022-12-31,2022,12,31,Business,SH,24.468,6.139,2.046,14.324,24.468,33.000,3.647,6.600,5,60.000,49.000,0.983,8.167,6,66.667,51.000,3.498,7.286,7,71.429,27.000,4.573,6.750,4,75.000,48.000,1.464,6.857,7,42.857,50.000,3.185,7.143,7,71.429,55.000,2.268,7.857,7,85.714,11.000,3.512,3.667,3,0.000,0.000,NaN,NaN,0,NaN,51.000,2.430,7.286,7,42.857,55.000,3.579,7.857,7,85.714,58.000,1.496,8.286,7,71.429,15.000,5.000,5.000,3,33.333,59.000,2.299,8.429,7,71.429,27.000,2.062,6.750,4,50.000,0.000,NaN,NaN,0,NaN,28.0

In [28]:
df['cabin'].unique()

array(['Business', 'Economy', 'Premium Economy'], dtype=object)

Lets check the NPS curve now against the dashboard:

In [29]:
shp_bi = pd.read_excel('inputs/SH Premium.xlsx')
shp_bi

,Quarter,Month_Name,2019,2022,2023
0,Q1,Jan,38.240,44.246,35.708
1,Q1,Feb,46.748,34.647,48.818
2,Q1,Mar,40.184,38.806,35.570
3,Q2,Apr,36.272,32.042,45.796
4,Q2,May,46.937,27.681,44.219
5,Q2,Jun,18.644,25.831,40.074
6,Q3,Jul,33.755,28.475,41.470
7,Q3,Aug,35.874,36.682,48.652
8,Q3,Sep,33.372,28.024,NaN
9,Q4,Oct,43.783,44.776,NaN


In [30]:
data=df[(df['date_flight_local'].dt.year.isin([2019, 2022, 2023])) & (df['cabin'] == 'Business') & (df['haul'] == 'SH')][['NPS', 'NPS_weighted', 'date_flight_local']]


In [31]:
# Pivot the dataframe
pivot_df = data.pivot_table(index=data['date_flight_local'].dt.month, columns=data['date_flight_local'].dt.year, values=['NPS', 'NPS_weighted'], aggfunc='mean')

# Rename the columns
pivot_df.columns = [f"{year}" for _, year in pivot_df.columns]

# Reset the index
pivot_df.reset_index(inplace=True)


pivot_df.rename(columns={'date_flight_local': 'Month_Name'}, inplace=True)

In [32]:
pivot_df

,Month_Name,2019,2022,2023,2019,2022,2023
0,1,36.304,44.081,36.123,36.304,44.081,36.123
1,2,47.428,34.056,48.867,47.428,34.056,48.867
2,3,38.790,39.974,35.117,38.790,39.974,35.117
3,4,38.884,29.625,45.239,38.884,29.710,45.239
4,5,49.752,26.300,44.470,49.752,26.300,44.470
5,6,21.055,25.731,39.890,21.055,25.731,39.890
6,7,36.255,27.727,41.576,36.255,27.727,41.576
7,8,38.939,37.320,49.433,38.939,37.320,49.433
8,9,34.135,27.318,37.593,34.135,27.318,37.593
9,10,44.181,44.769,43.658,44.181,44.769,43.658


In [33]:
df.to_csv('outputs/daily_NPS_5ch_adjusted_complete', index=False)

#### Touchpoints' satisfaction  adjustment for the change of survey

In [34]:
df=pd.read_csv('outputs/daily_NPS_5ch_adjusted_complete')
df.head()

,date_flight_local,year,month,day,cabin,haul,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
0,2019-01-01,2019,1,1,Business,LH,93.137,1.534,0.000,1.534,93.137,24.000,1.000,8.000,3,66.667,22.000,1.528,7.333,3,33.333,16.000,4.041,5.333,3,33.333,5.000,3.536,2.500,2,0.000,23.000,1.155,7.667,3,33.333,25.000,0.577,8.333,3,100.000,26.000,0.577,8.667,3,100.000,8.000,NaN,8.000,1,100.000,14.000,2.828,7.000,2,50.000,25.000,1.155,8.333,3,66.667,27.000,1.000,9.000,3,100.000,18.000,0.000,9.000,2,100.000,0.000,NaN,NaN,0,NaN,29.000,0.577,9.667,3,100.000,23.000,1.528,7.667,3,66.667,0.000,NaN,NaN,0,NaN,19.000,1.528,6.333,3,33.333,61.132,82.979,76.596
1,2019-01-01,2019,1,1,Business,LH,93.137,1.534,0.000,1.534,93.137,24.000,1.000,8.000,3,66.667,22.000,1.528,7.333,3,33.333,16.000,4.041,5.333,3,33.333,5.000,3.536,2.500,2,0.000,23.000,1.155,7.667,3,33.333,25.000,0.577,8.333,3,100.000,26.000,0.577,8.667,3,100.000,8.000,NaN,8.000,1,100.000,14.000,2.828,7.000,2,50.000,25.000,1.155,8.333,3,66.667,27.000,1.000,9.000,3,100.000,18.000,0.000,9.000,2,100.000,0.000,NaN,NaN,0,NaN,29.000,0.577,9.667,3,100.000,23.000,1.528,7.667,3,66.667,0.000,NaN,NaN,0,NaN,19.000,1.528,6.333,3,33.333,61.132,82.979,76.596
2,2019-01-02,2019,1,2,Business,LH,23.137,3.067,1.534,5.112,23.137,71.000,2.713,7.889,9,55.556,90.000,1.491,9.000,10,80.000,73.000,1.126,9.125,8,87.500,63.000,2.872,7.000,9,66.667,89.000,1.287,8.900,10,80.000,82.000,2.440,8.200,10,80.000,77.000,1.590,8.556,9,88.889,12.000,2.160,3.000,4,0.000,54.000,3.000,6.000,9,33.333,69.000,3.071,6.900,10,50.000,95.000,0.850,9.500,10,100.000,81.000,1.663,8.100,10,60.000,7.000,NaN,7.000,1,0.000,90.000,1.563,9.000,10,9

In [35]:
df['date_flight_local']=pd.to_datetime(df['date_flight_local'])

In [36]:
import pandas as pd
from io import StringIO

#Flight Booking
data = """
Touchpoint
IB Plus loyalty program
Boarding
Ease of contact by phone
IFE
Connections experience
In flight food and beverage
Airport security
Wi-Fi
Lounge
Arrivals experience
Check-in
Journey preparation support
Pilot's announcements
Aircraft interior
Cabin Crew
Punctuality
Response provided to the issue
"""
#5.0
data2 = """
Global adjustment
12.7
8.2
7.0
6.9
5.7
5.6
5.2
5.2
5.1
4.5
3.3
3.2
2.5
2.4
1.1
-0.2
-3.1
"""
#3.6
data3 = """
SH_Business
16.1
4.0
0.8
1.0
3.3
4.0
1.5
10.2
1.5
3.1
-1.2
4.8
-0.7
0.5
-3.0
-2.5
-2.6
"""
#7.5
data4 = """
LH_Business
11.3
5.5
11.4
5.0
12.6
3.3
14.0
2.3
8.2
6.2
0.8
7.9
1.2
2.1
3.6
1.2
3.4
"""
#4.6
data5 = """
SH_Economy
12.7
8.1
5.2
3.4
4.2
5.6
3.8
6.5
6.7
3.8
3.2
3.0
2.3
2.7
1.5
-0.8
-3.8
"""
#4.6
data5 = """
SH_Premium Economy
12.7
8.1
5.2
3.4
4.2
5.6
3.8
6.5
6.7
3.8
3.2
3.0
2.3
2.7
1.5
-0.8
-3.8
"""
#7.4
data6 = """
LH_Economy
11.2
10.5
10.1
7.4
10.0
8.6
9.6
1.8
9.8
7.5
7.0
3.6
3.5
4.4
3.9
1.7
-1.1
"""
#7.4
data7 = """
LH_Premium Economy
11.2
10.5
10.1
7.4
10.0
8.6
9.6
1.8
9.8
7.5
7.0
3.6
3.5
4.4
3.9
1.7
-1.1
"""

# Create StringIO objects to simulate file-like input
data_stream = StringIO(data)
data2_stream = StringIO(data2)
data3_stream = StringIO(data3)
data4_stream = StringIO(data4)
data5_stream = StringIO(data5)
data6_stream = StringIO(data6)
data7_stream = StringIO(data7)

# Create DataFrames
ta_df = pd.DataFrame({'Touchpoint': data_stream.getvalue().split('\n')[1:]})
ta_df['Global adjustment'] = data2_stream.getvalue().split('\n')[1:]
ta_df['SH_Business'] = data3_stream.getvalue().split('\n')[1:]
ta_df['LH_Business'] = data4_stream.getvalue().split('\n')[1:]
ta_df['SH_Economy'] = data5_stream.getvalue().split('\n')[1:]
ta_df['LH_Economy'] = data6_stream.getvalue().split('\n')[1:]
ta_df['LH_Premium Economy'] = data7_stream.getvalue().split('\n')[1:]

# Display the DataFrame
ta_df=ta_df.drop(0)
ta_df=ta_df.drop(18)
ta_df


,Touchpoint,Global adjustment,SH_Business,LH_Business,SH_Economy,LH_Economy,LH_Premium Economy
1,IB Plus loyalty program,12.7,16.1,11.3,12.7,11.2,11.2
2,Boarding,8.2,4.0,5.5,8.1,10.5,10.5
3,Ease of contact by phone,7.0,0.8,11.4,5.2,10.1,10.1
4,IFE,6.9,1.0,5.0,3.4,7.4,7.4
5,Connections experience,5.7,3.3,12.6,4.2,10.0,10.0
6,In flight food and beverage,5.6,4.0,3.3,5.6,8.6,8.6
7,Airport security,5.2,1.5,14.0,3.8,9.6,9.6
8,Wi-Fi,5.2,10.2,2.3,6.5,1.8,1.8
9,Lounge,5.1,1.5,8.2,6.7,9.8,9.8
10,Arrivals experience,4.5,3.1,6.2,3.8,7.5,7.5


In [37]:
satisfaction_mapping = {
    'IB Plus loyalty program': 'loy_200_loyalty_programme_satisfaction',
    'Boarding': 'pfl_500_boarding_satisfaction',
    'Ease of contact by phone': 'img_310_ease_contact_phone_satisfaction',
    'IFE': 'ifl_500_ife_satisfaction',
    'Connections experience': 'con_100_connections_satisfaction',
    'In flight food and beverage': 'ifl_400_food_drink_satisfaction',
    'Airport security': 'pfl_200_security_satisfaction',
    'Wi-Fi': 'ifl_600_wifi_satisfaction',
    'Lounge': 'pfl_300_lounge_satisfaction',
    #'Flight Booking': 'bkg_100_booking_satisfaction',
    'Arrivals experience': 'arr_100_arrivals_satisfaction',
    'Check-in': 'pfl_100_checkin_satisfaction',
    'Journey preparation support': 'bkg_200_journey_preparation_satisfaction',
    "Pilot's announcements": 'ifl_200_flight_crew_annoucements_satisfaction',
    'Aircraft interior': 'ifl_300_cabin_satisfaction',
    'Cabin Crew': 'ifl_100_cabin_crew_satisfaction',
    'Punctuality': 'pun_100_punctuality_satisfaction',
    'Response provided to the issue': 'inm_400_issues_response_satisfaction'
}

# Create a Series to use for mapping
satisfaction_series = pd.Series(satisfaction_mapping)

# Replace touchpoints with their satisfaction equivalents in the original DataFrame
ta_df['Touchpoint'] = ta_df['Touchpoint'].map(satisfaction_series)


#Flip the dataframe to match the NPS one:
flipped_df = ta_df.transpose()
flipped_df.columns = flipped_df.iloc[0]
ta_df = flipped_df.drop(flipped_df.index[0])
ta_df


Touchpoint,loy_200_loyalty_programme_satisfaction,pfl_500_boarding_satisfaction,img_310_ease_contact_phone_satisfaction,ifl_500_ife_satisfaction,con_100_connections_satisfaction,ifl_400_food_drink_satisfaction,pfl_200_security_satisfaction,ifl_600_wifi_satisfaction,pfl_300_lounge_satisfaction,arr_100_arrivals_satisfaction,pfl_100_checkin_satisfaction,bkg_200_journey_preparation_satisfaction,ifl_200_flight_crew_annoucements_satisfaction,ifl_300_cabin_satisfaction,ifl_100_cabin_crew_satisfaction,pun_100_punctuality_satisfaction,inm_400_issues_response_satisfaction
Global adjustment,12.7,8.2,7.0,6.9,5.7,5.6,5.2,5.2,5.1,4.5,3.3,3.2,2.5,2.4,1.1,-0.2,-3.1
SH_Business,16.1,4.0,0.8,1.0,3.3,4.0,1.5,10.2,1.5,3.1,-1.2,4.8,-0.7,0.5,-3.0,-2.5,-2.6
LH_Business,11.3,5.5,11.4,5.0,12.6,3.3,14.0,2.3,8.2,6.2,0.8,7.9,1.2,2.1,3.6,1.2,3.4
SH_Economy,12.7,8.1,5.2,3.4,4.2,5.6,3.8,6.5,6.7,3.8,3.2,3.0,2.3,2.7,1.5,-0.8,-3.8
LH_Economy,11.2,10.5,10.1,7.4,10.0,8.6,9.6,1.8,9.8,7.5,7.0,3.6,3.5,4.4,3.9,1.7,-1.1
LH_Premium Economy,11.2,10.5,10.1,7.4,10.0,8.6,9.6,1.8,9.8,7.5,7.0,3.6,3.5,4.4,3.9,1.7,-1.1


In [38]:
df.describe()

,year,month,day,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
count,16816.000,16816.000,16816.000,16816.000,16816.000,16816.000,16816.000,16816.000,16816.000,16318.000,16766.000,16816.000,16766.000,16816.000,16416.000,16812.000,16816.000,16812.000,16816.000,16410.000,16810.000,16816.000,16810.000,16816.000,12964.000,14794.000,16816.000,14794.000,16816.000,16428.000,16814.000,16816.000,16814.000,16816.000,16428.000,16816.000,16816.000,16816.000,16816.000,16406.000,16806.000,16816.000,16806.000,16816.000,11822.000,13852.000,16816.000,13852.000,16816.000,13498.000,15014.000,16816.000,15014.000,16816.000,16298.000,16730.000,16816.000,16730.000,16816.000,16428.000,16810.000,16816.000,16810.000,16816.000,16394.000,16808.000,16816.000,16808.000,16816.000,12930.000,14884.000,16816.000,14884.000,16816.000,16434.000,16816.000,16816.000,16816.000,16816.000,15340.000,16204.000,16816.000,16204.000,16816.000,10338.000,13302.000,16816.000,13302.000,16816.000,16130.000,16698.000,16816.000,16698.000,16816.000,16816.000,16816.000
mean,2020.969,6.385,15.721,30.816,33.488,14.489,62.686,30.815,442.417,2.584,7.565,58.944,66.273,529.279,2.261,7.991,66.530,73.126,546.902,1.928,8.245,66.153,76.300,58.497,2.041,7.892,7.377,69.952,529.851,2.148,7.901,67.250,70.298,534.869,2.036,8.022,67.449,71.965,547.298,1.877,8.333,65.990,76.731,71.906,3.313,5.063,13.182,32.502,142.888,2.301,7.433,18.417,63.517,303.448,2.593,6.843,44.947,52.003,550.038,2.087,8.204,66.351,75.330,531.132,2.066,7.985,66.411,71.806,148.229,2.576,7.392,20.154,64.067,562.177,2.142,8.545,68.265,82.122,222.630,2.132,7.435,30.324,58.754,26.202,3.050,3.187,8.953,14.637,191.145,3.038

In [39]:
original_df=df.copy()
adjustment_df=ta_df.drop("Global adjustment")

# Convert the values in the adjustment_df to numeric
adjustment_df = adjustment_df.apply(pd.to_numeric, errors='coerce')

# Filter out the rows where the year is 2023
adjusted_years = original_df['date_flight_local'].dt.year != 2023
adjusted_df = original_df[adjusted_years].copy()

# Add the adjustments to the corresponding touchpoint columns
for index, row in adjustment_df.iterrows():
    haul, cabin = index.split('_')
    mask = (adjusted_df['haul'] == haul) & (adjusted_df['cabin'] == cabin)
    for touchpoint_col in adjustment_df.columns:
        adjusted_df[touchpoint_col].fillna(0, inplace=True) 
        adjusted_df.loc[mask, touchpoint_col] += row[touchpoint_col]
        
# Concatenate adjusted_df with values from 2023
adjusted_df = pd.concat([adjusted_df, original_df[~adjusted_years]])




In [40]:
adjusted_df.describe()

,year,month,day,NPS,promoters_weight,detractors_weight,monthly_weight,NPS_weighted,bkg_200_journey_preparation_sum,bkg_200_journey_preparation_std,bkg_200_journey_preparation_mean,bkg_200_journey_preparation_not_nulls,bkg_200_journey_preparation_satisfaction,pfl_100_checkin_sum,pfl_100_checkin_std,pfl_100_checkin_mean,pfl_100_checkin_not_nulls,pfl_100_checkin_satisfaction,pfl_200_security_sum,pfl_200_security_std,pfl_200_security_mean,pfl_200_security_not_nulls,pfl_200_security_satisfaction,pfl_300_lounge_sum,pfl_300_lounge_std,pfl_300_lounge_mean,pfl_300_lounge_not_nulls,pfl_300_lounge_satisfaction,pfl_500_boarding_sum,pfl_500_boarding_std,pfl_500_boarding_mean,pfl_500_boarding_not_nulls,pfl_500_boarding_satisfaction,ifl_300_cabin_sum,ifl_300_cabin_std,ifl_300_cabin_mean,ifl_300_cabin_not_nulls,ifl_300_cabin_satisfaction,ifl_200_flight_crew_annoucements_sum,ifl_200_flight_crew_annoucements_std,ifl_200_flight_crew_annoucements_mean,ifl_200_flight_crew_annoucements_not_nulls,ifl_200_flight_crew_annoucements_satisfaction,ifl_600_wifi_sum,ifl_600_wifi_std,ifl_600_wifi_mean,ifl_600_wifi_not_nulls,ifl_600_wifi_satisfaction,ifl_500_ife_sum,ifl_500_ife_std,ifl_500_ife_mean,ifl_500_ife_not_nulls,ifl_500_ife_satisfaction,ifl_400_food_drink_sum,ifl_400_food_drink_std,ifl_400_food_drink_mean,ifl_400_food_drink_not_nulls,ifl_400_food_drink_satisfaction,ifl_100_cabin_crew_sum,ifl_100_cabin_crew_std,ifl_100_cabin_crew_mean,ifl_100_cabin_crew_not_nulls,ifl_100_cabin_crew_satisfaction,arr_100_arrivals_sum,arr_100_arrivals_std,arr_100_arrivals_mean,arr_100_arrivals_not_nulls,arr_100_arrivals_satisfaction,con_100_connections_sum,con_100_connections_std,con_100_connections_mean,con_100_connections_not_nulls,con_100_connections_satisfaction,pun_100_punctuality_sum,pun_100_punctuality_std,pun_100_punctuality_mean,pun_100_punctuality_not_nulls,pun_100_punctuality_satisfaction,loy_200_loyalty_programme_sum,loy_200_loyalty_programme_std,loy_200_loyalty_programme_mean,loy_200_loyalty_programme_not_nulls,loy_200_loyalty_programme_satisfaction,inm_400_issues_response_sum,inm_400_issues_response_std,inm_400_issues_response_mean,inm_400_issues_response_not_nulls,inm_400_issues_response_satisfaction,img_310_ease_contact_phone_sum,img_310_ease_contact_phone_std,img_310_ease_contact_phone_mean,img_310_ease_contact_phone_not_nulls,img_310_ease_contact_phone_satisfaction,load_factor,otp15_landing,otp15_takeoff
count,16816.000,16816.000,16816.000,16816.000,16816.000,16816.000,16816.000,16816.000,16816.000,16318.000,16766.000,16816.000,16816.000,16816.000,16416.000,16812.000,16816.000,16816.000,16816.000,16410.000,16810.000,16816.000,16816.000,16816.000,12964.000,14794.000,16816.000,16698.000,16816.000,16428.000,16814.000,16816.000,16816.000,16816.000,16428.000,16816.000,16816.000,16816.000,16816.000,16406.000,16806.000,16816.000,16816.000,16816.000,11822.000,13852.000,16816.000,16816.000,16816.000,13498.000,15014.000,16816.000,16800.000,16816.000,16298.000,16730.000,16816.000,16816.000,16816.000,16428.000,16810.000,16816.000,16816.000,16816.000,16394.000,16808.000,16816.000,16816.000,16816.000,12930.000,14884.000,16816.000,16814.000,16816.000,16434.000,16816.000,16816.000,16816.000,16816.000,15340.000,16204.000,16816.000,16816.000,16816.000,10338.000,13302.000,16816.000,16762.000,16816.000,16130.000,16698.000,16816.000,16796.000,16816.000,16816.000,16816.000
mean,2020.969,6.385,15.721,30.816,33.488,14.489,62.686,30.815,442.417,2.584,7.565,58.944,69.817,529.279,2.261,7.991,66.530,75.830,546.902,1.928,8.245,66.153,82.512,58.497,2.041,7.892,7.377,67.879,529.851,2.148,7.901,67.250,76.595,534.869,2.036,8.022,67.449,74.260,547.298,1.877,8.333,65.990,78.279,71.906,3.313,5.063,13.182,30.526,142.888,2.301,7.433,18.417,60.696,303.448,2.593,6.843,44.947,56.649,550.038,2.087,8.204,66.351,76.894,531.132,2.066,7.985,66.411,76.342,148.229,2.576,7.392,20.154,63.222,562.177,2.142,8.545,68.265,82.309,222.630,2.132,7.435,30.324,66.877,26.202,3.050,3.187,8.953,10.728,191.145,3.038

In [41]:
df_kpis[(df_kpis['year']==2019)  & (df_kpis['cabin']== 'Economy') & (df_kpis['haul']== 'SH')]

,year,month,day,haul,cabin,load_factor,otp15_landing,otp15_takeoff
192,2019,6,8,SH,Economy,83.802,79.891,86.685
200,2019,11,9,SH,Economy,82.538,93.949,95.541
207,2019,10,15,SH,Economy,81.881,86.344,90.308
212,2019,9,9,SH,Economy,87.712,87.118,91.485
217,2019,8,9,SH,Economy,88.672,77.405,83.221
...,...,...,...,...,...,...,...,...
17522,2019,8,22,SH,Economy,83.628,84.028,86.111
17534,2019,7,24,SH,Economy,86.828,87.942,90.644
17745,2019,2,4,SH,Economy,78.099,91.325,92.289
17753,2019,3,20,SH,Economy,83.624,83.871,93.088


In [42]:
check_dates_df=df[(df['year']==2022)  & (df['cabin']== 'Business') & (df['haul']== 'SH')]

In [43]:
# Create a date range covering the expected time period
start_date = check_dates_df['date_flight_local'].min()
end_date = check_dates_df['date_flight_local'].max()
expected_dates = pd.date_range(start_date, end_date)

# Find the missing dates using set operations
missing_dates = expected_dates.difference(check_dates_df['date_flight_local'])

# Print the missing dates
print("Missing dates:")
print(missing_dates)

Missing dates:
DatetimeIndex(['2022-02-14', '2022-03-16', '2022-03-17', '2022-11-30'], dtype='datetime64[ns]', freq=None)


In [44]:
adjusted_df.to_csv('outputs/daily_NPS_5ch_Touchpoints_adjusted_complete', index=False)